## QuantGAN for Risk Management

Here, we apply QuantGAN to calculate potential future loss from a purchase of a stock.
To find this, we calculate the differences between the purchase price of the stock and a q-percentile of a distribution of the stock price on each business date until a specified future date. As a measure of potential futures loss we take the maximum loss across the time interval.

To find the distributions of the future stock price, we first train the QuantGAN to obtain the stock price dynamics and then simulate the paths. The main idea here is instead of long full training of the model, we are going to use Transfer Learning principle and load the generator to the pretrained model (which was fully trained on 10 years of S&P500 price). After that, we train model only for a small number of batches on the historical price of the given stock going up to 2 years back (which takes considerably less time).

In [58]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

from preprocess.acf import *
from preprocess.gaussianize import *

from tensorflow.random import normal
from tensorflow.keras.models import load_model
import yfinance as yf
from model.tf_gan import GAN
from model.tf_tcn import *

generator = load_model(f"/Users/andreypak/Downloads/temporalCN-main/trained/trained_generator_SP500_daily")

In [66]:
def CalculateVaR(ticker, purchaseDate, NDays, NSim=500, q=0.05, n_batches = 10):
    purchaseDate = datetime.strptime(purchaseDate,'%Y-%m-%d')
    if not bool(len(pd.bdate_range(purchaseDate, purchaseDate))):
        return print(f"{purchaseDate} is not a business date!")

    df = yf.download(ticker, start=purchaseDate - timedelta(days=730) , end=purchaseDate+timedelta(days=1))
    ts = df['Adj Close']
    if len(ts)<500:
        return print("Price history is not long enough!")

    receptive_field_size = 127
    fixed_filters = 80
    block_size = 2
    batch_size = 64

    standardScaler1 = StandardScaler()
    standardScaler2 = StandardScaler()
    gaussianize = Gaussianize()

    log_returns = np.log(ts/ts.shift(1))[1:].to_numpy().reshape(-1, 1)
    log_returns_preprocessed = standardScaler2.fit_transform(gaussianize.fit_transform(standardScaler1.fit_transform(log_returns)))


    discriminator = TCN([receptive_field_size, 1],fixed_filters=100)

    gan = GAN(discriminator, generator, 2 * receptive_field_size - 1, lr_d=1e-4, lr_g=3e-5)
    log_returns_rolled = rolling_window(log_returns_preprocessed, receptive_field_size)
    data = np.expand_dims(np.moveaxis(log_returns_rolled, 0,1), 1).astype('float32')
    gan.train(data, batch_size, n_batches)


    noise = normal([NSim, 1, NDays+receptive_field_size-1, 3])
    y = generator(noise).numpy().squeeze()
    y = (y - y.mean(axis=0))/y.std(axis=0)
    y = standardScaler2.inverse_transform(y)
    y = np.array([gaussianize.inverse_transform(np.expand_dims(x, 1)) for x in y]).squeeze()
    y = standardScaler1.inverse_transform(y)

    S0 = df.loc[df.index == purchaseDate.strftime('%Y-%m-%d')]["Adj Close"].to_numpy().squeeze()
    S = np.exp(np.cumsum(np.insert(y, 0, np.ones(NSim)*np.log(S0), axis=1), axis=1))
    Q = np.quantile(S, q, axis=0)
    L = S0-Q

    
    print(f"The purchase price of stock is {S0}.")
    print(f"There is {q} probability that the loss on {ticker} will exceed {max(L)} if bought on {purchaseDate} and hold for {NDays}-day (business) period.")
    plt.plot(S.T, alpha=0.75)


# User Guide
User only needs to know:
- ticker name, 
- date of stock purchase,
- the horizon in business days for which user is planning to hold the stock.

optional:
- number of simulation paths,
- loss percentile,
- number of batches.

The output: 
- Stock purchase price,
- The potential future loss.  

In [69]:
CalculateVaR("CPNG", "2023-02-08", 30)

[*********************100%***********************]  1 of 1 completed
Price history is not long enough!
